# Spotify API

## Authentication and Authorization Tokens

Load client credentials from .env file.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

Use client credentials to obtain access token for Spotify API. 

See https://developer.spotify.com/documentation/web-api/tutorials/client-credentials-flow for details.

In [8]:
import requests
import time
import base64
import json
import os

TOKEN_ENDPOINT = 'https://accounts.spotify.com/api/token'
TOKEN_FILE = 'spotify_token.json'

class SpotifyAuthenticator:
    def __init__(self):
        self.access_token = None
        self.expires_at = 0
        self.load_token()
        
    def load_token(self):
        """Loads most recent access token."""
        if os.path.exists(TOKEN_FILE):
            with open(TOKEN_FILE, 'r') as file:
                data = json.load(file)
                self.access_token = data.get('access_token')
                self.expires_at = data.get('expires_at', 0)
        
    def save_token(self):
        """For access token persistence."""
        with open(TOKEN_FILE, 'w') as file:
            json.dump({'access_token': self.access_token, 'expires_at': self.expires_at}, file)
        
    def get_token(self):
        """Returns access token or refreshes token if expired."""
        if not self.access_token or time.time() >= self.expires_at:
            self.refresh_token()
        return self.access_token
    
    def refresh_token(self):
        """Requests access token."""
        print('Refreshing access token...')
        
        credentials = f'{CLIENT_ID}:{CLIENT_SECRET}'
        
        # converts credentials: string -> bytes, bytes encode to Base64, bytes -> string
        encoded_credentials = base64.b64encode(credentials.encode()).decode()
        
        headers = {
            'Authorization': f'Basic {encoded_credentials}',
            'Content-Type': 'application/x-www-form-urlencoded'
        }
        data = {'grant_type': 'client_credentials'}
        
        response = requests.post(TOKEN_ENDPOINT, headers=headers, data=data)
        
        if response.status_code == 200:
            token_data = response.json()
            self.access_token = token_data['access_token']
            self.expires_at = time.time() + token_data['expires_in'] - 60
            self.save_token()
            print('Access token refreshed successfully!')
        else:
            raise Exception(f'Could not refresh access token: {response.status_code}, {response.text}')
            
spotify_authenticator = SpotifyAuthenticator()

## Querying

In [12]:
def spotify_search(query, category='track'):
    token = spotify_authenticator.get_token()
    url = 'https://api.spotify.com/v1/search'
    headers = {'Authorization': f'Bearer {token}'}
    params = {'q': query, 'type': category, 'limit': 1}
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f'Error {response.status_code}: {response.text}')
        
result = spotify_search('When You Sleep')
result

{'tracks': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=1&query=When%20You%20Sleep&type=track',
  'limit': 1,
  'next': 'https://api.spotify.com/v1/search?offset=1&limit=1&query=When%20You%20Sleep&type=track',
  'offset': 0,
  'previous': None,
  'total': 1000,
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3G3Gdm0ZRAOxLrbyjfhii5'},
       'href': 'https://api.spotify.com/v1/artists/3G3Gdm0ZRAOxLrbyjfhii5',
       'id': '3G3Gdm0ZRAOxLrbyjfhii5',
       'name': 'my bloody valentine',
       'type': 'artist',
       'uri': 'spotify:artist:3G3Gdm0ZRAOxLrbyjfhii5'}],
     'available_markets': ['CA', 'US'],
     'external_urls': {'spotify': 'https://open.spotify.com/album/3GH4IiI6jQAIvnHVdb5FB6'},
     'href': 'https://api.spotify.com/v1/albums/3GH4IiI6jQAIvnHVdb5FB6',
     'id': '3GH4IiI6jQAIvnHVdb5FB6',
     'images': [{'height': 640,
       'width': 640,
       'url': 'https://i.scdn.co/image/ab6

## Processing